In [1]:
import os
import pickle
file_path = "vector_store.pkl"
file_name="paper.pdf"
vector_file_path = os.path.join("vectorStore",str.replace(file_name, 'pdf','pkl'))

In [8]:
model="smollm"
from langchain_ollama import OllamaLLM
model=OllamaLLM(model=model)

In [10]:
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader(file_name)
pages=loader.load_and_split()
# print(f"the number of pages loaded from the document are: {len(pages)}\n\nContent of the first page:{'-'*20} \n\n{pages[0]}")

the number of pages loaded from the document are: 9

Content of the first page:-------------------- 

page_content='RRCSNet:Recurrent Residual Convolutional Neural  
Network based cloud segmentation from satellite  
images  
 
 
 
 
 
 
 
 
 
 
 
 
Abstract— A crucial role in remote sensing applications,  
particularly in satellite imagery analysis, is played by cloud  
segmentation.  In this paper,  RRCSNet,  a novel  Recurrent  
Residual Convolutional Neural Network designed for accurate  
and efficient  cloud  segmentation.  The architecture  combines  the 
strengths  of recurrent  neural  networks  and residual  
convolutional  neural  networks  and enhance  feature  
representation.  The model  learns  sequential  patterns  in satellite  
data through recurrent connections, and residual connections  
help train  deeper  networks  by addressing  the vanishing  gradient  
problem.  Experimental  results  on Landsat  satellite  datasets  
prove the higher performance of RRCSNet compa

In [11]:
from langchain.prompts import PromptTemplate

template = """ Answer the question asked on the basis of the context provided.
If you can not find the answer then just say "answer is not available in the given text
context : {context}
Question : {question}
"""
prompt= PromptTemplate.from_template(template=template)
# print(prompt.format(context="Here is the context", question="Here is the question"))


 Answer the question asked on the basis of the context provided.
If you can not find the answer then just say "answer is not available in the given text
context : Here is the context
Question : Here is the question



In [14]:
from langchain_ollama import OllamaEmbeddings
embeddings=OllamaEmbeddings(model="smollm")

In [16]:

from langchain_community.vectorstores import DocArrayInMemorySearch
vector_store=DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)

In [17]:
retriever=vector_store.as_retriever()
# retriever.invoke("What is the result of the study")

[Document(metadata={'source': 'paper.pdf', 'page': 2}, page_content='dimensionality  of features.  The activation  function  ReLU  is \ndefined  as \nReLU(X)=max(0,X)  \nTo incorporate low -level details of cloud with high -level  \nfeatures from feature maps in different scales full scale skip  \nconnections are added in the encoder path.  The input image  \nundergoes multiple layers of convolutions and pooling in the  \nencoding  path,  with recurrent  residual  blocks  enhancing  \nfeature  extraction  at each stage.  These  features  are then \nprocessed  through  a series of  residual  blocks,  enhancing  \nfeature learning and retention. The output from the residual  \nblocks is fed into recurrent layers, which capture long -range  \ndependencies  and refine  the segmentation  mask.  The recurrent  \nconnections are built using GRU units that process the input  \nfeature maps recurrently. The GRU gates help in selectively  \nremembering and forgetting information, thus enabling t

In [18]:
from operator import itemgetter

chain = ({
    "context":itemgetter("question") | retriever,
    "question":itemgetter("question")
    }|
    prompt|model
    )

In [19]:
print(chain.invoke({"question":"What is the result"}))

The answer is: atmospheric  conditions.


In [20]:
# questions=[
#     "What is the title of the paper", 
#     "How many authors are there in the paper",
#     "what is the plateform used to run the experiment", 
#     "summerize the methodology"
# ]

# for question in questions:
#     print(f"question: {question}")
#     print(f"Answer:{chain.invoke({'question':question})}")
#     print()

question: What is the title of the paper
Answer:The title of the paper is "Deep Convolutional Neural Networks for Multiscale Cloud Detection in Satellite Imagery"

question: How many authors are there in the paper
Answer:The paper "A Recurrent Residual Convolutional Neural Network for Cloud Segmentation from Satellite Imagery" by Xue et al. (2020) has 5 authors listed on its author page.

To find all authors of a paper, you can use the following steps:

1. Go to the paper's author page and look for the "Authors" section.
2. Check if there are multiple authors listed. If so, separate them by commas (e.g., "Xue, Y., Zhang, H., Li, J., & Chen, X.").
3. If there is only one author listed, you can stop here and use the paper's title or abstract to find more information about the research.

question: what is the plateform used to run the experiment
Answer:The plateform used to run the experiment in this paper is likely a high-performance computing (HPC) cluster or a distributed computing env